In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
# optional
import seaborn as sns; sns.set()
import math
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import GradientBoostingClassifier
from scipy.sparse.linalg import svds
from functools import partial
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute


train = pd.read_table("../Data_competition/X_train.txt", sep=',',header=None)
test = pd.read_table("../Data_competition/X_test.txt",sep=',',header=None)
cible=pd.read_table("../Data_competition/y_train.txt",sep=',',header=None)
header=pd.read_csv("../Data_competition/header.csv",sep=',',header=None)

feature_name=header.drop(header.columns[[2]], axis=1)
feature_name=feature_name.drop(feature_name.index[[56,49,30,20,14,0,38]])[3]
feature_name
feature_name2=header.drop(header.columns[[2]], axis=1)
feature_name2=feature_name2.drop(feature_name2.index[[56,49,30,20,14,0,38]])[1]
                                 
train_via_def=train
train_via_def.columns=feature_name2
test_via_def=test
test_via_def.columns=feature_name2

frames=[train_via_def,test_via_def]
alldata= pd.concat(frames,keys=['train','test'])



In [68]:
alldata['x51']=alldata['x51']-2011 

In [69]:
alldata.head()

1              x1        x2        x3        x4        x5        x6        x7  \
train 0  0.491722  1.264287 -0.158577 -0.150768  1.087627  0.157166 -1.053879   
      1 -0.249351  0.046088 -0.040435  0.105421 -1.098919  0.289237 -0.338154   
      2  0.986824 -0.189678  0.059693  0.010147  0.777260 -1.051633 -1.011747   
      3 -0.265990 -0.517960 -0.114526 -0.073818  1.889234 -0.161217 -0.947844   
      4  0.199900 -0.095447  0.263825  0.040790 -1.125166 -1.318912 -0.887661   

1              x8        x9       x10       x11       x12       x13       x14  \
train 0 -1.116961       NaN       NaN       NaN       NaN       NaN -0.235758   
      1  0.129297  0.807200       NaN       NaN       NaN       NaN       NaN   
      2 -0.756690       NaN  0.874877 -0.273440  0.086508  0.688904  0.011214   
      3 -0.650268 -0.712451       NaN  0.126789  0.596843       NaN       NaN   
      4 -1.202686       NaN       NaN       NaN       NaN       NaN       NaN   

1        x15  x16       x17  x18       x19       x20       x21       x22  \
train 0  NaN  NaN  0.107888  NaN       NaN -0.047326 -0.399256  0.024927   
      1  NaN  NaN       NaN  NaN       NaN  0.008015 -0.736507       NaN   
      2  NaN  NaN -0.327590  NaN  0.154479       NaN  0.956582       NaN   
      3  NaN  NaN       NaN  NaN -0.081760       NaN       NaN       NaN   
      4  NaN  NaN       NaN  NaN       NaN       NaN       NaN       NaN   

1             x23       x24       x25       x26       x27       x28  x29  \
train 0       NaN       NaN       NaN       NaN       NaN  0.164409  NaN   
      1 -0.417189       NaN       NaN       NaN -0.657171       NaN  NaN   
      2       NaN       NaN  1.195586       NaN       NaN       NaN  NaN   
      3 -0.455399 -0.077836       NaN  0.271222       NaN       NaN  NaN   
      4       NaN  2.624526  0.983099 -1.652295       NaN       NaN  NaN   

1             x30       x31       x32       x33       x34  x35       x36  \
train 0       NaN  0.066269       NaN       NaN       NaN  NaN       NaN   
      1       NaN       NaN  0.445685       NaN       NaN  NaN       NaN   
      2       NaN  0.286557       NaN       NaN -1.309222  NaN  0.296849   
      3  0.465727       NaN       NaN       NaN -1.189501  NaN       NaN   
      4       NaN       NaN -2.776458 -0.118833       NaN  NaN       NaN   

1             x37       x38       x39       x40       x41       x42       x43  \
train 0       NaN -0.262937       NaN       NaN  0.887558       NaN       NaN   
      1 -0.329228       NaN       NaN       NaN       NaN  0.468655       NaN   
      2  0.264740       NaN  0.513461       NaN  0.404907 -0.271227       NaN   
      3       NaN       NaN       NaN  0.117178       NaN       NaN -0.378151   
      4       NaN       NaN       NaN       NaN -0.424390 -0.568337 -0.180425   

1             x44       x45      x46       x47       x48       x49       x50  \
train 0       NaN       NaN      NaN  5.284473       NaN       NaN       NaN   
      1  0.527339 -0.291376      NaN       NaN       NaN       NaN       NaN   
      2  0.995816  0.004617  0.48018       NaN -0.105363       NaN       NaN   
      3       NaN       NaN      NaN       NaN       NaN -0.755457 -0.526293   
      4       NaN       NaN      NaN       NaN       NaN       NaN -0.535741   

1        x51  
train 0  0.0  
      1  0.0  
      2  0.0  
      3  0.0  
      4  0.0

In [70]:
alldata2=alldata.drop(['x51'],axis=1)

In [71]:
alldata_norm=alldata2.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

In [72]:
alldata_norm.head()

1              x1        x2        x3        x4        x5        x6        x7  \
train 0  0.026572  0.012514 -0.002939 -0.021647  0.055704  0.027360 -0.024279   
      1 -0.021013  0.002316 -0.000783 -0.000444 -0.073290  0.058090 -0.010460   
      2  0.058363  0.000342  0.001044 -0.008330  0.037394 -0.253903 -0.023465   
      3 -0.022081 -0.002407 -0.002135 -0.015279  0.102995 -0.046721 -0.022232   
      4  0.007834  0.001131  0.004769 -0.005793 -0.074838 -0.316094 -0.021070   

1              x8        x9       x10       x11       x12       x13       x14  \
train 0 -0.014648       NaN       NaN       NaN       NaN       NaN -0.078616   
      1 -0.000868  0.090096       NaN       NaN       NaN       NaN       NaN   
      2 -0.010665       NaN  0.060124 -0.027510  0.008149  0.008214  0.002151   
      3 -0.009488 -0.070325       NaN  0.043907  0.040231       NaN       NaN   
      4 -0.015596       NaN       NaN       NaN       NaN       NaN       NaN   

1        x15  x16       x17  x18       x19       x20       x21       x22  \
train 0  NaN  NaN  0.005950  NaN       NaN -0.005781 -0.026789  0.002417   
      1  NaN  NaN       NaN  NaN       NaN -0.004200 -0.051252       NaN   
      2  NaN  NaN -0.006243  NaN  0.004817       NaN  0.071560       NaN   
      3  NaN  NaN       NaN  NaN -0.001683       NaN       NaN       NaN   
      4  NaN  NaN       NaN  NaN       NaN       NaN       NaN       NaN   

1             x23       x24       x25       x26       x27       x28  x29  \
train 0       NaN       NaN       NaN       NaN       NaN  0.001394  NaN   
      1 -0.010925       NaN       NaN       NaN -0.050315       NaN  NaN   
      2       NaN       NaN  0.098701       NaN       NaN       NaN  NaN   
      3 -0.012167  0.000546       NaN  0.003091       NaN       NaN  NaN   
      4       NaN  0.084384  0.081333 -0.014411       NaN       NaN  NaN   

1             x30       x31       x32       x33       x34  x35       x36  \
train 0       NaN  0.002956       NaN       NaN       NaN  NaN       NaN   
      1       NaN       NaN  0.001642       NaN       NaN  NaN       NaN   
      2       NaN  0.006321       NaN       NaN -0.015586  NaN  0.011556   
      3  0.015196       NaN       NaN       NaN -0.014255  NaN       NaN   
      4       NaN       NaN -0.003342 -0.001808       NaN  NaN       NaN   

1             x37       x38       x39      x40       x41       x42       x43  \
train 0       NaN -0.007831       NaN      NaN  0.080819       NaN       NaN   
      1 -0.018985       NaN       NaN      NaN       NaN -0.004714       NaN   
      2  0.014226       NaN  0.026238      NaN  0.035114 -0.007065       NaN   
      3       NaN       NaN       NaN  0.00078       NaN       NaN -0.027836   
      4       NaN       NaN       NaN      NaN -0.043417 -0.008009 -0.013172   

1             x44       x45       x46       x47       x48       x49       x50  
train 0       NaN       NaN       NaN  0.041048       NaN       NaN       NaN  
      1  0.015329 -0.005804       NaN       NaN       NaN       NaN       NaN  
      2  0.032095 -0.000139 -0.000863       NaN -0.000153       NaN       NaN  
      3       NaN       NaN       NaN       NaN       NaN -0.011925 -0.026717  
      4       NaN       NaN       NaN       NaN       NaN       NaN -0.026982

In [73]:
alldata_norm2=alldata_norm.drop(['x1','x2','x3','x4','x5','x6','x7','x8'],axis=1)

In [74]:
alldata_filled_softimpute_2 = SoftImpute().complete(alldata_norm2)

[SoftImpute] Max Singular Value of X_init = 5.562690
[SoftImpute] Iter 1: observed MAE=0.001315 rank=42
[SoftImpute] Iter 2: observed MAE=0.001320 rank=42
[SoftImpute] Iter 3: observed MAE=0.001324 rank=42
[SoftImpute] Iter 4: observed MAE=0.001323 rank=41
[SoftImpute] Iter 5: observed MAE=0.001319 rank=41
[SoftImpute] Iter 6: observed MAE=0.001318 rank=41
[SoftImpute] Iter 7: observed MAE=0.001318 rank=41
[SoftImpute] Iter 8: observed MAE=0.001319 rank=41
[SoftImpute] Iter 9: observed MAE=0.001320 rank=41
[SoftImpute] Iter 10: observed MAE=0.001321 rank=41
[SoftImpute] Iter 11: observed MAE=0.001322 rank=41
[SoftImpute] Iter 12: observed MAE=0.001324 rank=41
[SoftImpute] Iter 13: observed MAE=0.001325 rank=41
[SoftImpute] Iter 14: observed MAE=0.001326 rank=41
[SoftImpute] Iter 15: observed MAE=0.001327 rank=41
[SoftImpute] Iter 16: observed MAE=0.001328 rank=41
[SoftImpute] Iter 17: observed MAE=0.001328 rank=41
[SoftImpute] Iter 18: observed MAE=0.001329 rank=41
[SoftImpute] Iter 19

In [75]:
colonneToFill=['x'+str(j) for j in range(9,51)]

In [76]:
datacomplete=pd.DataFrame(alldata_filled_softimpute_2,columns=colonneToFill)

In [77]:
datacomplete.head()

,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50
0,-0.002936,0.004534,0.008377,0.001229,0.000405,-0.078616,-0.003140,0.001587,0.005950,-0.003561,0.000862,-0.005781,-0.026789,0.002417,-0.000205,0.005156,-0.005498,-0.004848,-0.013960,0.001394,-0.006511,0.005562,0.002956,-0.009864,0.000047,0.006085,0.001404,0.013265,0.020625,-0.007831,-0.005282,-0.006843,0.080819,-0.000887,-0.009944,-0.002083,-0.003324,-0.007373,0.041048,-0.000338,-0.001336,-0.003047
1,0.090096,-0.006332,-0.006238,-0.001019,-0.000263,-0.001617,-0.002787,0.000370,0.001375,0.001089,0.002598,-0.004200,-0.051252,-0.007113,-0.010925,0.000169,-0.022551,-0.001602,-0.050315,-0.000955,0.001014,0.002073,0.000004,0.001642,0.002040,-0.000556,-0.002598,-0.002992,-0.018985,-0.001913,-0.002725,-0.001596,-0.009722,-0.004714,-0.004210,0.015329,-0.005804,0.000804,-0.000678,-0.005889,0.001021,0.001138
2,-0.010902,0.060124,-0.027510,0.008149,0.008214,0.002151,0.004097,0.001562,-0.006243,0.003743,0.004817,0.007269,0.071560,0.007335,0.006235,0.003187,0.098701,0.003747,0.046652,0.000997,-0.008060,0.005187,0.006321,0.011186,0.004804,-0.015586,0.001296,0.011556,0.014226,0.016278,0.026238,-0.000918,0.035114,-0.007065,0.006209,0.032095,-0.000139,-0.000863,0.002077,-0.000153,-0.006280,-0.004165
3,-0.070325,0.011641,0.043907,0.040231,0.005362,-0.000524,0.000227,0.000839,-0.001338,-0.001047,-0.001683,0.001411,-0.003179,0.003347,-0.012167,0.000546,-0.000377,0.003091,-0.000264,0.000537,-0.004402,0.015196,0.006836,0.010497,0.000388,-0.014255,0.002482,0.002169,0.003756,0.000820,0.004217,0.000780,0.002817,-0.001798,-0.027836,0.001721,-0.000338,0.000882,0.002501,0.002201,-0.011925,-0.026717
4,0.002843,-0.002251,0.004237,0.003530,0.002055,-0.000006,0.007039,0.000204,0.011326,-0.000505,-0.005408,-0.041798,0.023654,0.043896,0.009895,0.084384,0.081333,-0.014411,0.023075,-0.001312,0.000073,0.004977,0.000139,-0.003342,-0.001808,0.004922,0.004000,-0.007202,-0.010272,0.007453,0.004994,-0.009110,-0.043417,-0.008009,-0.013172,-0.000394,0.002581,0.001235,0.000796,0.004348,0.001110,-0.026982


In [89]:
datacomplete.index=alldata_norm.index

In [97]:
datacomplete[['x1','x2','x3','x4','x5','x6','x7','x8']]=alldata_norm[['x1','x2','x3','x4','x5','x6','x7','x8']]
datacomplete['x51']=alldata['x51']

In [98]:
datacomplete.head()

x9       x10       x11       x12       x13       x14       x15  \
train 0 -0.002936  0.004534  0.008377  0.001229  0.000405 -0.078616 -0.003140   
      1  0.090096 -0.006332 -0.006238 -0.001019 -0.000263 -0.001617 -0.002787   
      2 -0.010902  0.060124 -0.027510  0.008149  0.008214  0.002151  0.004097   
      3 -0.070325  0.011641  0.043907  0.040231  0.005362 -0.000524  0.000227   
      4  0.002843 -0.002251  0.004237  0.003530  0.002055 -0.000006  0.007039   

              x16       x17       x18       x19       x20       x21       x22  \
train 0  0.001587  0.005950 -0.003561  0.000862 -0.005781 -0.026789  0.002417   
      1  0.000370  0.001375  0.001089  0.002598 -0.004200 -0.051252 -0.007113   
      2  0.001562 -0.006243  0.003743  0.004817  0.007269  0.071560  0.007335   
      3  0.000839 -0.001338 -0.001047 -0.001683  0.001411 -0.003179  0.003347   
      4  0.000204  0.011326 -0.000505 -0.005408 -0.041798  0.023654  0.043896   

              x23       x24       x25       x26       x27       x28       x29  \
train 0 -0.000205  0.005156 -0.005498 -0.004848 -0.013960  0.001394 -0.006511   
      1 -0.010925  0.000169 -0.022551 -0.001602 -0.050315 -0.000955  0.001014   
      2  0.006235  0.003187  0.098701  0.003747  0.046652  0.000997 -0.008060   
      3 -0.012167  0.000546 -0.000377  0.003091 -0.000264  0.000537 -0.004402   
      4  0.009895  0.084384  0.081333 -0.014411  0.023075 -0.001312  0.000073   

              x30       x31       x32       x33       x34       x35       x36  \
train 0  0.005562  0.002956 -0.009864  0.000047  0.006085  0.001404  0.013265   
      1  0.002073  0.000004  0.001642  0.002040 -0.000556 -0.002598 -0.002992   
      2  0.005187  0.006321  0.011186  0.004804 -0.015586  0.001296  0.011556   
      3  0.015196  0.006836  0.010497  0.000388 -0.014255  0.002482  0.002169   
      4  0.004977  0.000139 -0.003342 -0.001808  0.004922  0.004000 -0.007202   

              x37       x38       x39       x40       x41       x42       x43  \
train 0  0.020625 -0.007831 -0.005282 -0.006843  0.080819 -0.000887 -0.009944   
      1 -0.018985 -0.001913 -0.002725 -0.001596 -0.009722 -0.004714 -0.004210   
      2  0.014226  0.016278  0.026238 -0.000918  0.035114 -0.007065  0.006209   
      3  0.003756  0.000820  0.004217  0.000780  0.002817 -0.001798 -0.027836   
      4 -0.010272  0.007453  0.004994 -0.009110 -0.043417 -0.008009 -0.013172   

              x44       x45       x46       x47       x48       x49       x50  \
train 0 -0.002083 -0.003324 -0.007373  0.041048 -0.000338 -0.001336 -0.003047   
      1  0.015329 -0.005804  0.000804 -0.000678 -0.005889  0.001021  0.001138   
      2  0.032095 -0.000139 -0.000863  0.002077 -0.000153 -0.006280 -0.004165   
      3  0.001721 -0.000338  0.000882  0.002501  0.002201 -0.011925 -0.026717   
      4 -0.000394  0.002581  0.001235  0.000796  0.004348  0.001110 -0.026982   

               x1        x2        x3        x4        x5        x6        x7  \
train 0  0.026572  0.012514 -0.002939 -0.021647  0.055704  0.027360 -0.024279   
      1 -0.021013  0.002316 -0.000783 -0.000444 -0.073290  0.058090 -0.010460   
      2  0.058363  0.000342  0.001044 -0.008330  0.037394 -0.253903 -0.023465   
      3 -0.022081 -0.002407 -0.002135 -0.015279  0.102995 -0.046721 -0.022232   
      4  0.007834  0.001131  0.004769 -0.005793 -0.074838 -0.316094 -0.021070   

               x8  x51  
train 0 -0.014648  0.0  
      1 -0.000868  0.0  
      2 -0.010665  0.0  
      3 -0.009488  0.0  
      4 -0.015596  0.0

In [101]:
train_nan_SoftImpute=datacomplete.ix['train']
test_nan_SoftImpute=datacomplete.ix['test']

In [102]:
tuned_parameters = [{'loss': ['deviance'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300,500],'learning_rate':[0.1,0.3,0.5]}]

clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=10,verbose=2)
clf.fit(train_nan_SoftImpute, cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 
[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance, total=   0.2s
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance, total=   0.2s
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 
[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance, total=   0.2s
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 
[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance, total=   0.2s
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 
[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance, total=   0.2s
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 
[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance, total=   0.2s
[CV] learning_rate=0.1, max_features=5, max_depth=3, n_estimators=100, loss=deviance 
[CV]  learning_rate=0.1, max_features=5, max_depth=3, n_estimator

[Parallel(n_jobs=1)]: Done 1440 out of 1440 | elapsed: 29.9min finished


Best parameters set found on development set:

{'learning_rate': 0.3, 'max_features': 5, 'max_depth': 5, 'n_estimators': 300, 'loss': 'deviance'}

Grid scores on development set:

0.942 (+/-0.048) for {'learning_rate': 0.1, 'max_features': 5, 'max_depth': 3, 'n_estimators': 100, 'loss': 'deviance'}
0.951 (+/-0.044) for {'learning_rate': 0.1, 'max_features': 5, 'max_depth': 3, 'n_estimators': 300, 'loss': 'deviance'}
0.952 (+/-0.044) for {'learning_rate': 0.1, 'max_features': 5, 'max_depth': 3, 'n_estimators': 500, 'loss': 'deviance'}
0.944 (+/-0.049) for {'learning_rate': 0.1, 'max_features': 10, 'max_depth': 3, 'n_estimators': 100, 'loss': 'deviance'}
0.951 (+/-0.044) for {'learning_rate': 0.1, 'max_features': 10, 'max_depth': 3, 'n_estimators': 300, 'loss': 'deviance'}
0.957 (+/-0.035) for {'learning_rate': 0.1, 'max_features': 10, 'max_depth': 3, 'n_estimators': 500, 'loss': 'deviance'}
0.945 (+/-0.044) for {'learning_rate': 0.1, 'max_features': 20, 'max_depth': 3, 'n_estimators': 1

In [103]:
clf.best_score_

0.95750000000000002

In [104]:
model=GradientBoostingClassifier(max_depth=5,loss='deviance',n_estimators=300,learning_rate=0.3,max_features=5)

In [105]:
model.fit(train_nan_SoftImpute,cible[0].values)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.3, loss='deviance', max_depth=5,
              max_features=5, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [106]:
y_pred_clf=model.predict(test_nan_SoftImpute)

index=range(len(y_pred_clf))
data=np.transpose([index,y_pred_clf])
a = pd.DataFrame(data=data,columns=['Id','Prediction'])
a.to_csv("submission7.csv", sep=",",index=False)  

## Strategy svd train and svd train test and take test

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
# optional
import seaborn as sns; sns.set()
import math
from sklearn.model_selection import GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import GradientBoostingClassifier
from scipy.sparse.linalg import svds
from functools import partial
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute


Using TensorFlow backend.


In [4]:
train = pd.read_table("../Data_competition/X_train.txt", sep=',',header=None)
test = pd.read_table("../Data_competition/X_test.txt",sep=',',header=None)
cible=pd.read_table("../Data_competition/y_train.txt",sep=',',header=None)
header=pd.read_csv("../Data_competition/header.csv",sep=',',header=None)

feature_name=header.drop(header.columns[[2]], axis=1)
feature_name=feature_name.drop(feature_name.index[[56,49,30,20,14,0,38]])[3]
feature_name
feature_name2=header.drop(header.columns[[2]], axis=1)
feature_name2=feature_name2.drop(feature_name2.index[[56,49,30,20,14,0,38]])[1]
                                 
train_via_def=train
train_via_def.columns=feature_name2
test_via_def=test
test_via_def.columns=feature_name2

frames=[train_via_def,test_via_def]
alldata= pd.concat(frames,keys=['train','test'])
alldata['x51']=alldata['x51']-2011 

In [5]:
train=alldata.ix['train']
test=alldata.ix['test']

In [7]:
train.head()

1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51
0,0.491722,1.264287,-0.158577,-0.150768,1.087627,0.157166,-1.053879,-1.116961,NaN,NaN,NaN,NaN,NaN,-0.235758,NaN,NaN,0.107888,NaN,NaN,-0.047326,-0.399256,0.024927,NaN,NaN,NaN,NaN,NaN,0.164409,NaN,NaN,0.066269,NaN,NaN,NaN,NaN,NaN,NaN,-0.262937,NaN,NaN,0.887558,NaN,NaN,NaN,NaN,NaN,5.284473,NaN,NaN,NaN,0.0
1,-0.249351,0.046088,-0.040435,0.105421,-1.098919,0.289237,-0.338154,0.129297,0.807200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008015,-0.736507,NaN,-0.417189,NaN,NaN,NaN,-0.657171,NaN,NaN,NaN,NaN,0.445685,NaN,NaN,NaN,NaN,-0.329228,NaN,NaN,NaN,NaN,0.468655,NaN,0.527339,-0.291376,NaN,NaN,NaN,NaN,NaN,0.0
2,0.986824,-0.189678,0.059693,0.010147,0.777260,-1.051633,-1.011747,-0.756690,NaN,0.874877,-0.273440,0.086508,0.688904,0.011214,NaN,NaN,-0.327590,NaN,0.154479,NaN,0.956582,NaN,NaN,NaN,1.195586,NaN,NaN,NaN,NaN,NaN,0.286557,NaN,NaN,-1.309222,NaN,0.296849,0.264740,NaN,0.513461,NaN,0.404907,-0.271227,NaN,0.995816,0.004617,0.48018,NaN,-0.105363,NaN,NaN,0.0
3,-0.265990,-0.517960,-0.114526,-0.073818,1.889234,-0.161217,-0.947844,-0.650268,-0.712451,NaN,0.126789,0.596843,NaN,NaN,NaN,NaN,NaN,NaN,-0.081760,NaN,NaN,NaN,-0.455399,-0.077836,NaN,0.271222,NaN,NaN,NaN,0.465727,NaN,NaN,NaN,-1.189501,NaN,NaN,NaN,NaN,NaN,0.117178,NaN,NaN,-0.378151,NaN,NaN,NaN,NaN,NaN,-0.755457,-0.526293,0.0
4,0.199900,-0.095447,0.263825,0.040790,-1.125166,-1.318912,-0.887661,-1.202686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.624526,0.983099,-1.652295,NaN,NaN,NaN,NaN,NaN,-2.776458,-0.118833,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.424390,-0.568337,-0.180425,NaN,NaN,NaN,NaN,NaN,NaN,-0.535741,0.0


In [9]:
train2=train.drop(['x51'],axis=1)

In [13]:
train_2_norm=train2.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

In [16]:
train_2_softimpute = SoftImpute().complete(train_2_norm)

[SoftImpute] Max Singular Value of X_init = 14.595494
[SoftImpute] Iter 1: observed MAE=0.004846 rank=50
[SoftImpute] Iter 2: observed MAE=0.004863 rank=50
[SoftImpute] Iter 3: observed MAE=0.004877 rank=50
[SoftImpute] Iter 4: observed MAE=0.004889 rank=50
[SoftImpute] Iter 5: observed MAE=0.004899 rank=50
[SoftImpute] Iter 6: observed MAE=0.004907 rank=50
[SoftImpute] Iter 7: observed MAE=0.004915 rank=50
[SoftImpute] Iter 8: observed MAE=0.004921 rank=50
[SoftImpute] Iter 9: observed MAE=0.004927 rank=50
[SoftImpute] Iter 10: observed MAE=0.004932 rank=50
[SoftImpute] Iter 11: observed MAE=0.004937 rank=50
[SoftImpute] Iter 12: observed MAE=0.004941 rank=50
[SoftImpute] Iter 13: observed MAE=0.004945 rank=50
[SoftImpute] Iter 14: observed MAE=0.004949 rank=50
[SoftImpute] Iter 15: observed MAE=0.004952 rank=50
[SoftImpute] Iter 16: observed MAE=0.004955 rank=50
[SoftImpute] Iter 17: observed MAE=0.004958 rank=50
[SoftImpute] Iter 18: observed MAE=0.004959 rank=49
[SoftImpute] Iter 1

In [17]:
train_2_complete=pd.DataFrame(train_2_softimpute,columns=train2.columns)

In [18]:
train_2_complete.head()

1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50
0,0.030146,0.014774,-0.008783,-0.016354,0.080429,0.042962,-0.022163,-0.029362,0.045060,0.036827,0.059160,0.044438,0.037434,-0.104992,0.006013,-0.001314,0.006566,-0.002369,-0.006777,-0.004623,-0.041150,0.004292,0.015052,0.008375,-0.003222,0.001143,-0.058667,0.011274,-0.018054,0.038240,0.016047,-0.002039,-0.003566,-0.024875,0.006665,0.022201,0.055156,-0.021711,-0.044636,-0.042705,0.112222,-0.014961,-0.053559,-0.044333,0.019519,0.034273,0.488989,0.045189,-0.010821,-0.018408
1,-0.018435,0.001180,-0.002551,0.005480,-0.088480,0.081887,-0.008344,0.001867,0.091589,-0.021653,-0.018943,0.022463,-0.004442,-0.000565,-0.005034,0.001108,-0.005450,0.003528,-0.007586,-0.002903,-0.076705,-0.001278,-0.037985,0.004963,-0.027370,-0.004152,-0.065741,-0.005487,0.013563,0.014416,0.002090,0.033020,0.000302,0.016461,0.001763,-0.019991,-0.039129,0.010238,0.006934,0.009278,-0.020408,-0.002510,0.001462,0.058233,-0.023894,-0.002798,0.006100,0.001462,0.012278,-0.001994
2,0.062602,-0.001451,0.002731,-0.002640,0.056453,-0.313305,-0.021349,-0.020334,-0.038999,0.066192,-0.045137,0.009710,0.029452,0.000800,0.010419,0.002114,-0.010182,0.005609,0.005987,0.004608,0.101792,0.026218,0.028732,0.013162,0.095969,0.023940,0.069167,0.024265,-0.069032,0.010417,0.048204,0.024385,0.016221,-0.278323,0.007807,0.034296,0.027354,0.054272,0.027668,0.026572,0.049267,-0.013705,0.010626,0.115187,-0.003412,0.004825,-0.014113,-0.002057,-0.018782,-0.008053
3,-0.019526,-0.005114,-0.006460,-0.009796,0.142352,-0.050874,-0.020116,-0.017668,-0.070678,0.038799,0.031489,0.041792,0.026841,0.001134,0.001315,0.002200,-0.004468,0.002256,-0.000740,0.004680,-0.013331,0.013790,-0.042152,-0.000439,-0.004630,0.016443,-0.006785,0.026854,-0.070229,0.014196,0.047777,0.037674,0.009958,-0.253206,0.003010,0.022015,0.026012,0.008290,0.000340,0.010242,0.023730,-0.000260,-0.025987,-0.012510,-0.004872,0.003248,-0.004874,0.007406,-0.069619,-0.024832
4,0.011016,-0.000399,0.013498,-0.000028,-0.090508,-0.392079,-0.018954,-0.031510,-0.008286,-0.018684,-0.056038,-0.139244,-0.039437,-0.003880,-0.010813,-0.003345,0.090412,-0.002128,0.002413,-0.089552,0.021771,-0.049952,-0.022991,0.083399,0.078602,-0.112669,0.025303,-0.049894,0.118578,-0.015884,-0.085901,-0.211897,-0.001313,0.050475,-0.029801,-0.052585,-0.019851,0.009856,-0.000736,-0.019887,-0.058903,-0.018201,-0.011324,0.030447,0.021289,-0.025368,-0.000569,-0.012962,-0.007983,-0.025181


In [20]:
train_fill_svd=train_2_complete
train_fill_svd['x51']=train['x51']

In [21]:
train_fill_svd.head()

1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51
0,0.030146,0.014774,-0.008783,-0.016354,0.080429,0.042962,-0.022163,-0.029362,0.045060,0.036827,0.059160,0.044438,0.037434,-0.104992,0.006013,-0.001314,0.006566,-0.002369,-0.006777,-0.004623,-0.041150,0.004292,0.015052,0.008375,-0.003222,0.001143,-0.058667,0.011274,-0.018054,0.038240,0.016047,-0.002039,-0.003566,-0.024875,0.006665,0.022201,0.055156,-0.021711,-0.044636,-0.042705,0.112222,-0.014961,-0.053559,-0.044333,0.019519,0.034273,0.488989,0.045189,-0.010821,-0.018408,0.0
1,-0.018435,0.001180,-0.002551,0.005480,-0.088480,0.081887,-0.008344,0.001867,0.091589,-0.021653,-0.018943,0.022463,-0.004442,-0.000565,-0.005034,0.001108,-0.005450,0.003528,-0.007586,-0.002903,-0.076705,-0.001278,-0.037985,0.004963,-0.027370,-0.004152,-0.065741,-0.005487,0.013563,0.014416,0.002090,0.033020,0.000302,0.016461,0.001763,-0.019991,-0.039129,0.010238,0.006934,0.009278,-0.020408,-0.002510,0.001462,0.058233,-0.023894,-0.002798,0.006100,0.001462,0.012278,-0.001994,0.0
2,0.062602,-0.001451,0.002731,-0.002640,0.056453,-0.313305,-0.021349,-0.020334,-0.038999,0.066192,-0.045137,0.009710,0.029452,0.000800,0.010419,0.002114,-0.010182,0.005609,0.005987,0.004608,0.101792,0.026218,0.028732,0.013162,0.095969,0.023940,0.069167,0.024265,-0.069032,0.010417,0.048204,0.024385,0.016221,-0.278323,0.007807,0.034296,0.027354,0.054272,0.027668,0.026572,0.049267,-0.013705,0.010626,0.115187,-0.003412,0.004825,-0.014113,-0.002057,-0.018782,-0.008053,0.0
3,-0.019526,-0.005114,-0.006460,-0.009796,0.142352,-0.050874,-0.020116,-0.017668,-0.070678,0.038799,0.031489,0.041792,0.026841,0.001134,0.001315,0.002200,-0.004468,0.002256,-0.000740,0.004680,-0.013331,0.013790,-0.042152,-0.000439,-0.004630,0.016443,-0.006785,0.026854,-0.070229,0.014196,0.047777,0.037674,0.009958,-0.253206,0.003010,0.022015,0.026012,0.008290,0.000340,0.010242,0.023730,-0.000260,-0.025987,-0.012510,-0.004872,0.003248,-0.004874,0.007406,-0.069619,-0.024832,0.0
4,0.011016,-0.000399,0.013498,-0.000028,-0.090508,-0.392079,-0.018954,-0.031510,-0.008286,-0.018684,-0.056038,-0.139244,-0.039437,-0.003880,-0.010813,-0.003345,0.090412,-0.002128,0.002413,-0.089552,0.021771,-0.049952,-0.022991,0.083399,0.078602,-0.112669,0.025303,-0.049894,0.118578,-0.015884,-0.085901,-0.211897,-0.001313,0.050475,-0.029801,-0.052585,-0.019851,0.009856,-0.000736,-0.019887,-0.058903,-0.018201,-0.011324,0.030447,0.021289,-0.025368,-0.000569,-0.012962,-0.007983,-0.025181,0.0


In [22]:
tuned_parameters = [{'loss': ['deviance','exponential'],
                    'max_features':[5,10,20,50],
                    'max_depth':[3,5,10,30],
                    'n_estimators':[100,300]}]

clf = GridSearchCV(GradientBoostingClassifier(), tuned_parameters, cv=10,verbose=1)
clf.fit(train_fill_svd, cible[0].values)


print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()

print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))
print()

Fitting 10 folds for each of 64 candidates, totalling 640 fits


[Parallel(n_jobs=1)]: Done 640 out of 640 | elapsed: 19.3min finished


Best parameters set found on development set:

{'n_estimators': 300, 'loss': 'deviance', 'max_features': 50, 'max_depth': 5}

Grid scores on development set:

0.943 (+/-0.048) for {'n_estimators': 100, 'loss': 'deviance', 'max_features': 5, 'max_depth': 3}
0.950 (+/-0.047) for {'n_estimators': 300, 'loss': 'deviance', 'max_features': 5, 'max_depth': 3}
0.945 (+/-0.046) for {'n_estimators': 100, 'loss': 'deviance', 'max_features': 10, 'max_depth': 3}
0.951 (+/-0.044) for {'n_estimators': 300, 'loss': 'deviance', 'max_features': 10, 'max_depth': 3}
0.947 (+/-0.045) for {'n_estimators': 100, 'loss': 'deviance', 'max_features': 20, 'max_depth': 3}
0.953 (+/-0.047) for {'n_estimators': 300, 'loss': 'deviance', 'max_features': 20, 'max_depth': 3}
0.948 (+/-0.039) for {'n_estimators': 100, 'loss': 'deviance', 'max_features': 50, 'max_depth': 3}
0.954 (+/-0.041) for {'n_estimators': 300, 'loss': 'deviance', 'max_features': 50, 'max_depth': 3}
0.949 (+/-0.039) for {'n_estimators': 100, 'loss': 

In [23]:
clf.best_score_

0.95833333333333337

In [24]:
alldata2=alldata.drop(['x51'],axis=1)
alldata_norm=alldata2.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
alldata_filled_softimpute = SoftImpute().complete(alldata_norm)

[SoftImpute] Max Singular Value of X_init = 14.938213
[SoftImpute] Iter 1: observed MAE=0.003039 rank=49
[SoftImpute] Iter 2: observed MAE=0.003045 rank=49
[SoftImpute] Iter 3: observed MAE=0.003057 rank=49
[SoftImpute] Iter 4: observed MAE=0.003065 rank=48
[SoftImpute] Iter 5: observed MAE=0.003072 rank=48
[SoftImpute] Iter 6: observed MAE=0.003079 rank=48
[SoftImpute] Iter 7: observed MAE=0.003086 rank=48
[SoftImpute] Iter 8: observed MAE=0.003092 rank=48
[SoftImpute] Iter 9: observed MAE=0.003098 rank=48
[SoftImpute] Iter 10: observed MAE=0.003103 rank=48
[SoftImpute] Iter 11: observed MAE=0.003108 rank=48
[SoftImpute] Iter 12: observed MAE=0.003112 rank=48
[SoftImpute] Iter 13: observed MAE=0.003116 rank=48
[SoftImpute] Iter 14: observed MAE=0.003119 rank=48
[SoftImpute] Iter 15: observed MAE=0.003122 rank=47
[SoftImpute] Iter 16: observed MAE=0.003123 rank=47
[SoftImpute] Iter 17: observed MAE=0.003125 rank=47
[SoftImpute] Iter 18: observed MAE=0.003127 rank=47
[SoftImpute] Iter 1

In [41]:
colonnenames=['x'+str(j) for j in range(1,51)]

In [45]:
datacomplete=pd.DataFrame(alldata_filled_softimpute,columns=colonnenames)
datacomplete.index=alldata.index

In [46]:
datacomplete.head()

x1        x2        x3        x4        x5        x6        x7  \
train 0  0.026572  0.012514 -0.002939 -0.021647  0.055704  0.027360 -0.024279   
      1 -0.021013  0.002316 -0.000783 -0.000444 -0.073290  0.058090 -0.010460   
      2  0.058363  0.000342  0.001044 -0.008330  0.037394 -0.253903 -0.023465   
      3 -0.022081 -0.002407 -0.002135 -0.015279  0.102995 -0.046721 -0.022232   
      4  0.007834  0.001131  0.004769 -0.005793 -0.074838 -0.316094 -0.021070   

               x8        x9       x10       x11       x12       x13       x14  \
train 0 -0.014648 -0.004845  0.018160  0.022667  0.018572  0.002512 -0.078616   
      1 -0.000868  0.090096 -0.017003 -0.008325  0.005938 -0.003025 -0.001207   
      2 -0.010665 -0.016875  0.060124 -0.027510  0.008149  0.008214  0.002151   
      3 -0.009488 -0.070325  0.016365  0.043907  0.040231  0.009006 -0.002205   
      4 -0.015596  0.002112 -0.013278 -0.014481 -0.018145 -0.007824  0.000240   

              x15       x16       x17       x18       x19       x20       x21  \
train 0 -0.003680  0.002174  0.005950 -0.005389  0.001306 -0.005781 -0.026789   
      1 -0.002867  0.000688  0.002583  0.000797  0.000611 -0.004200 -0.051252   
      2  0.005303  0.000178 -0.006243  0.002711  0.004817  0.004545  0.071560   
      3  0.000488  0.000255 -0.002102 -0.001033 -0.001683  0.002721 -0.002433   
      4  0.007879 -0.001337  0.010431 -0.003936 -0.005566 -0.043131  0.028487   

              x22       x23       x24       x25       x26       x27       x28  \
train 0  0.002417  0.003169  0.009074 -0.005795 -0.004970 -0.018294  0.001394   
      1 -0.007976 -0.010925  0.002268 -0.025675 -0.000671 -0.050315 -0.000912   
      2  0.007220  0.007105  0.005131  0.098701  0.004724  0.060543  0.000108   
      3  0.003591 -0.012167  0.000546 -0.001168  0.003091  0.001289  0.000673   
      4  0.046239  0.010296  0.084384  0.081333 -0.014411  0.034251 -0.002544   

              x29       x30       x31       x32       x33       x34       x35  \
train 0 -0.007144  0.019054  0.002956 -0.007305  0.000321  0.003968  0.002114   
      1  0.001466  0.011217  0.002247  0.001642  0.000984 -0.000030 -0.001213   
      2 -0.004276  0.012132  0.006321  0.007804  0.007022 -0.015586  0.004155   
      3 -0.002810  0.015196  0.006198  0.008269  0.000323 -0.014255  0.003834   
      4  0.000687  0.008229  0.001188 -0.003342 -0.001808  0.001435  0.001567   

              x36       x37       x38       x39       x40       x41       x42  \
train 0  0.014392  0.028303 -0.007831 -0.005971 -0.007248  0.080819 -0.002760   
      1  0.001093 -0.018985 -0.000646 -0.001139 -0.001490 -0.012797 -0.004714   
      2  0.011556  0.014226  0.016901  0.026238  0.000988  0.035114 -0.007065   
      3 -0.007454  0.005487  0.000340  0.003527  0.000780  0.003552 -0.004412   
      4 -0.003847 -0.015245  0.010481  0.004417 -0.004463 -0.043417 -0.008009   

              x43       x44       x45       x46       x47       x48       x49  \
train 0 -0.015377 -0.006910 -0.003028 -0.007625  0.041048  0.002014 -0.003700   
      1 -0.003142  0.015329 -0.005804 -0.001562  0.004105 -0.002955 -0.001056   
      2  0.002219  0.032095 -0.000139 -0.000863  0.000667 -0.000153 -0.010890   
      3 -0.027836  0.000537  0.000399  0.002169 -0.006742  0.003476 -0.011925   
      4 -0.013172 -0.000632  0.005336 -0.006274  0.007749  0.003468  0.003528   

              x50  
train 0 -0.003924  
      1  0.002666  
      2 -0.016024  
      3 -0.026717  
      4 -0.026982

In [47]:
datacomplete['x51']=alldata['x51']

In [48]:
datacomplete.head()

x1        x2        x3        x4        x5        x6        x7  \
train 0  0.026572  0.012514 -0.002939 -0.021647  0.055704  0.027360 -0.024279   
      1 -0.021013  0.002316 -0.000783 -0.000444 -0.073290  0.058090 -0.010460   
      2  0.058363  0.000342  0.001044 -0.008330  0.037394 -0.253903 -0.023465   
      3 -0.022081 -0.002407 -0.002135 -0.015279  0.102995 -0.046721 -0.022232   
      4  0.007834  0.001131  0.004769 -0.005793 -0.074838 -0.316094 -0.021070   

               x8        x9       x10       x11       x12       x13       x14  \
train 0 -0.014648 -0.004845  0.018160  0.022667  0.018572  0.002512 -0.078616   
      1 -0.000868  0.090096 -0.017003 -0.008325  0.005938 -0.003025 -0.001207   
      2 -0.010665 -0.016875  0.060124 -0.027510  0.008149  0.008214  0.002151   
      3 -0.009488 -0.070325  0.016365  0.043907  0.040231  0.009006 -0.002205   
      4 -0.015596  0.002112 -0.013278 -0.014481 -0.018145 -0.007824  0.000240   

              x15       x16       x17       x18       x19       x20       x21  \
train 0 -0.003680  0.002174  0.005950 -0.005389  0.001306 -0.005781 -0.026789   
      1 -0.002867  0.000688  0.002583  0.000797  0.000611 -0.004200 -0.051252   
      2  0.005303  0.000178 -0.006243  0.002711  0.004817  0.004545  0.071560   
      3  0.000488  0.000255 -0.002102 -0.001033 -0.001683  0.002721 -0.002433   
      4  0.007879 -0.001337  0.010431 -0.003936 -0.005566 -0.043131  0.028487   

              x22       x23       x24       x25       x26       x27       x28  \
train 0  0.002417  0.003169  0.009074 -0.005795 -0.004970 -0.018294  0.001394   
      1 -0.007976 -0.010925  0.002268 -0.025675 -0.000671 -0.050315 -0.000912   
      2  0.007220  0.007105  0.005131  0.098701  0.004724  0.060543  0.000108   
      3  0.003591 -0.012167  0.000546 -0.001168  0.003091  0.001289  0.000673   
      4  0.046239  0.010296  0.084384  0.081333 -0.014411  0.034251 -0.002544   

              x29       x30       x31       x32       x33       x34       x35  \
train 0 -0.007144  0.019054  0.002956 -0.007305  0.000321  0.003968  0.002114   
      1  0.001466  0.011217  0.002247  0.001642  0.000984 -0.000030 -0.001213   
      2 -0.004276  0.012132  0.006321  0.007804  0.007022 -0.015586  0.004155   
      3 -0.002810  0.015196  0.006198  0.008269  0.000323 -0.014255  0.003834   
      4  0.000687  0.008229  0.001188 -0.003342 -0.001808  0.001435  0.001567   

              x36       x37       x38       x39       x40       x41       x42  \
train 0  0.014392  0.028303 -0.007831 -0.005971 -0.007248  0.080819 -0.002760   
      1  0.001093 -0.018985 -0.000646 -0.001139 -0.001490 -0.012797 -0.004714   
      2  0.011556  0.014226  0.016901  0.026238  0.000988  0.035114 -0.007065   
      3 -0.007454  0.005487  0.000340  0.003527  0.000780  0.003552 -0.004412   
      4 -0.003847 -0.015245  0.010481  0.004417 -0.004463 -0.043417 -0.008009   

              x43       x44       x45       x46       x47       x48       x49  \
train 0 -0.015377 -0.006910 -0.003028 -0.007625  0.041048  0.002014 -0.003700   
      1 -0.003142  0.015329 -0.005804 -0.001562  0.004105 -0.002955 -0.001056   
      2  0.002219  0.032095 -0.000139 -0.000863  0.000667 -0.000153 -0.010890   
      3 -0.027836  0.000537  0.000399  0.002169 -0.006742  0.003476 -0.011925   
      4 -0.013172 -0.000632  0.005336 -0.006274  0.007749  0.003468  0.003528   

              x50  x51  
train 0 -0.003924  0.0  
      1  0.002666  0.0  
      2 -0.016024  0.0  
      3 -0.026717  0.0  
      4 -0.026982  0.0

In [49]:
test_nan_SoftImpute=datacomplete.ix['test']

In [50]:
model=GradientBoostingClassifier(max_depth=5,loss='deviance',n_estimators=300,max_features=50)
model.fit(train_fill_svd,cible[0].values)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=50, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=300, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [51]:
y_pred_clf=model.predict(test_nan_SoftImpute)

index=range(len(y_pred_clf))
data=np.transpose([index,y_pred_clf])
a = pd.DataFrame(data=data,columns=['Id','Prediction'])
a.to_csv("submission8.csv", sep=",",index=False)  